In [2]:
import pandas as pd
import numpy as np
import glob
import os, os.path
#import geopy.distance

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from math import sin, cos, sqrt, atan2, radians

In [5]:
country_name = pd.read_excel("../../data/country_names.xlsx")

## Preparing the dataset for the plant wise regression
- the variables ending with "_Var" are the ones changing in different years
- currently doesn't include "Sub-Saharan Africa","Latin America & Caribbean","Middle East & North Africa" which include roughly 5 examples

### Fuel prices and reserves
- International coal reserve https://www.eia.gov/international/data/world/coal-and-coke/coal-reserves [30/6/2023]

In [6]:
# units in USD per million BTU
ng_price = pd.read_excel("../../data/global_BP_StatisticalReview/bp-stats-review-2022-all-data.xlsx",
                         sheet_name="Gas Prices ",skiprows=3)
ng_price = ng_price.rename(columns={"Unnamed: 0":"Year"})
ng_price = ng_price.iloc[1:-8,:-3]
ng_price["Year"] = ng_price["Year"].astype(int)
ng_price = ng_price.set_index("Year")
ng_price.head()

,Japan,Japan Korea Marker,Average German,UK,Netherlands TTF,US,Canada,OECD
Year,,,,,,,,
1984,5.1,-,3.995635,-,-,-,-,5
1985,5.234653,-,4.253587,-,-,-,-,4.751724
1986,4.10198,-,3.928619,-,-,-,-,2.574138
1987,3.352638,-,2.547503,-,-,-,-,3.094828
1988,3.344109,-,2.220359,-,-,-,-,2.562069


In [7]:
# USD per tonne
coal_price = pd.read_excel("../../data/global_BP_StatisticalReview/bp-stats-review-2022-all-data.xlsx",
                         sheet_name="Coal Prices",skiprows=1)
coal_price = coal_price.rename(columns={"US dollars per tonne":"Year"})
coal_price = coal_price.iloc[1:-6,:-5]
coal_price["Year"] = coal_price["Year"].astype(int)
coal_price = coal_price.set_index("Year")
coal_price.head()

,Northwest Europe marker price †,US Central Appalachian coal spot price index ‡,Japan steam spot CIF price †,China Qinhuangdao spot price*,Japan coking coal import CIF price,Japan steam coal import CIF price,Asian marker price †
Year,,,,,,,
1987,31.30,-,-,-,53.439167,41.281667,-
1988,39.94,-,-,-,55.064167,42.465833,-
1989,42.08,-,-,-,58.684167,48.862500,-
1990,43.48,31.591918,-,-,60.536667,50.814167,-
1991,42.80,29.010532,-,-,60.450000,50.295833,-


In [8]:
coal_r2p = pd.read_csv("../../data/global_BP_StatisticalReview/coal_reserve2production.csv")
ng_r2p = pd.read_csv("../../data/global_BP_StatisticalReview/gas_reserve2production.csv")
ng_r2p = ng_r2p.rename(columns={"Unnamed: 0":"Year"}).set_index("Year")

### whole dataset

In [9]:
reform_data = pd.read_csv("../_data_process/_all_temporal_power_reform.csv")
BP_data = pd.read_csv("../_data_process/_all_temporal_BPstats.csv")
world_dev_data =  pd.read_csv("../_data_process/_all_temporal_world_development.csv")

In [31]:
reform_2022 = reform_data.query("year==2021")
reform_2022["year"] = 2022
reform_data = pd.concat([reform_data,reform_2022])
reform_data

,country,year,source,R_IndepProducer,R_Private,R_Unbundle,R_WholeSale,R_IndepReg,R_Choice,R_Liberalization,R_Corp
0,Greece,1999.0,Erdogdu 2011,0,0,1,0,1,0,1,0
1,Greece,2000.0,Erdogdu 2011,0,0,1,0,1,0,1,1
2,Greece,2001.0,Erdogdu 2011,0,0,1,0,1,1,1,1
3,Greece,2002.0,Erdogdu 2011,0,1,1,0,1,1,1,1
4,Greece,2003.0,Erdogdu 2011,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
7087,"Gambia, The",2022.0,assumption,1,0,0,0,0,0,1,1
7095,Morocco,2022.0,assumption,1,1,1,0,0,0,1,1
7103,Burkina Faso,2022.0,assumption,1,0,0,0,0,0,1,1
7111,Jordan,2022.0,assumption,1,0,1,1,1,1,1,1


### Initial data

In [10]:
# plant-wise transitioned coal data
tran_data = pd.read_csv("../../data/global_GEM/analysis_plant/coal2gas_matching_combined.csv")
tran_data

,Unnamed: 0,GEM location ID,Latitude,Longitude,Plant name,Status,Start year,Retired year,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Country,Coal-to-gas conversion/replacement?,Capacity elec. (MW)
0,0,L103096,-37.544470,175.148805,Huntly power station,operating,1985,not found,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,New Zealand,N,500
1,1,L103096,-37.544470,175.148805,Huntly power station,operating,2004,not found,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,New Zealand,N,51
2,2,L103096,-37.544470,175.148805,Huntly power station,operating,2007,not found,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,New Zealand,N,403
3,3,L405108,-33.209894,151.542652,Colongra power station,operating,2009,not found,-33.212081,151.542216,Munmorah power station,retired,2012.0,1969,600.0,L100008,Australia,NaN,724
4,4,L405117,-32.202000,115.773000,Kwinana Newgen power station,operating,2008,not found,-32.198337,115.775155,Kwinana power station,retired,2015.0,1970,640.0,L100041,Australia,NaN,320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,9,L101975,49.223900,7.014900,Romerbrucke power station,operating,2005,not found,49.223900,7.014900,NaN,NaN,NaN,1965,53.0,NaN,Germany,replacement,74.0
433,10,L103381,54.035556,39.779444,Ryazanskaya GRES power station,operating,1984,not found,54.035556,39.779444,NaN,NaN,NaN,1973,1600.0,NaN,Russia,conversion,1600.0
434,11,L400790,53.073680,-0.859010,Staythorpe C power station,operating,2011,not found,53.073680,-0.859010,NaN,NaN,NaN,1950,720.0,NaN,China,replacement,1772.0
435,12,L103348,67.499648,64.018893,Vorkutinskaya-2 power station,operating,2021,not found,67.499648,64.018893,NaN,NaN,NaN,1975,165.0,NaN,China,conversion,165.0


In [11]:
# use the gas plant online year as the coal end year
# assuming transition happens instantly 
manual_index = tran_data[tran_data["Coal_EndYr"].isna()==True].index
tran_data.loc[manual_index,"Coal_EndYr"] = tran_data.loc[manual_index,"Start year"]
# coal end year not found 
# manually delete one with later start year 
tran_data = tran_data[tran_data["GEM location ID"]!="L103380"]
# edit the ones with existing gas start year
nf_index = tran_data[tran_data["Coal_EndYr"]=="not found"].index
tran_data.loc[nf_index,"Coal_EndYr"] = tran_data.loc[nf_index,"Start year"]

In [12]:
# unique coal data
tran_data_coal = tran_data[["Coal_Latitude","Coal_Longitude","Coal_Plant","Coal_Status","Coal_EndYr",
                        "Coal_StartYr","Coal_MW","Coal_TrackerLOC","Country"]]
tran_data_coal = tran_data_coal.drop_duplicates()
tran_data_coal["Transition"] = 1
tran_data_coal

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Country,Transition
0,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,New Zealand,1
3,-33.212081,151.542216,Munmorah power station,retired,2012.0,1969,600.0,L100008,Australia,1
4,-32.198337,115.775155,Kwinana power station,retired,2015.0,1970,640.0,L100041,Australia,1
7,-27.660536,152.812899,Swanbank-A power station,retired,2012.0,1966,396.0,L405135,Australia,1
8,-26.115776,28.194019,Kelvin power station,retired,2022.0,1957,180.0,L103425,South Africa,1
...,...,...,...,...,...,...,...,...,...,...
432,49.223900,7.014900,NaN,NaN,2005,1965,53.0,NaN,Germany,1
433,54.035556,39.779444,NaN,NaN,1984,1973,1600.0,NaN,Russia,1
434,53.073680,-0.859010,NaN,NaN,2011,1950,720.0,NaN,China,1
435,67.499648,64.018893,NaN,NaN,2021,1975,165.0,NaN,China,1


In [13]:
# retired only data
ret_only_coal = pd.read_csv("../../data/global_GEM/analysis_plant/coal2gas_retired_only.csv")
ret_only_coal["Transition"] = 0
ret_only_coal

,Coal_TrackerLOC,Coal_Latitude,Coal_Longitude,Country,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,geometry,Transition
0,L100031,-38.386550,144.182062,Australia,Anglesea power station,retired,2015.0,1969,160.0,POINT (144.1820619 -38.3865497),0
1,L100034,-38.271833,146.390879,Australia,Hazelwood power station,retired,2017.0,1964,1600.0,POINT (146.3908786 -38.2718333),0
2,L100033,-38.253970,146.413667,Australia,Energy Brix power station,retired,2014.0,1958,150.0,POINT (146.4136665 -38.2539704),0
3,L100195,-37.021800,-73.166300,Chile,Bocamina power station,retired,2022.0,1970,478.0,POINT (-73.16630000000001 -37.0218),0
4,L100042,-33.445875,116.307528,Australia,Muja power station,retired,2022.0,1966,440.0,POINT (116.3075275 -33.445875),0
...,...,...,...,...,...,...,...,...,...,...,...
816,L101851,62.027100,24.634100,Finland,Mänttä power station,retired,not found,1958,65.0,POINT (24.6341 62.0271),0
817,L101837,62.254911,21.326515,Finland,Kristiina power station,retired,2017.0,1983,242.0,POINT (21.326515 62.254911),0
818,L104489,64.730219,177.496925,Russia,Anadyrskaya power station,retired,2021.0,1986,60.0,POINT (177.496925 64.73021900000001),0
819,L104448,67.598725,33.423457,Russia,Apatitskaya CHPP power station,retired,2016.0,1959,72.0,POINT (33.423457 67.598725),0


### combine datasets 

In [14]:
pw_data_coal = pd.concat([tran_data_coal,ret_only_coal])
pw_data_coal = pd.merge(pw_data_coal,country_name[["GEM_Name","Region","Country Code"]],left_on="Country",right_on="GEM_Name")
pw_data_coal = pw_data_coal.drop(columns=["Country","geometry"])
pw_data_coal

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,Region,Country Code
0,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,1,New Zealand,East Asia & Pacific,NZL
1,-33.212081,151.542216,Munmorah power station,retired,2012.0,1969,600.0,L100008,1,Australia,East Asia & Pacific,AUS
2,-32.198337,115.775155,Kwinana power station,retired,2015.0,1970,640.0,L100041,1,Australia,East Asia & Pacific,AUS
3,-27.660536,152.812899,Swanbank-A power station,retired,2012.0,1966,396.0,L405135,1,Australia,East Asia & Pacific,AUS
4,-38.386550,144.182062,Anglesea power station,retired,2015.0,1969,160.0,L100031,0,Australia,East Asia & Pacific,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...
1077,48.552500,21.972625,Vojany I power station,retired,2010.0,1965,440.0,L103410,0,Slovakia,Europe & Central Asia,SVK
1078,48.698823,18.533458,Novaky power station,retired,2015.0,1957,252.0,L103415,0,Slovakia,Europe & Central Asia,SVK
1079,49.059218,18.907019,Martinska power station,retired,2020.0,1962,32.0,L103416,0,Slovakia,Europe & Central Asia,SVK
1080,59.353269,18.100901,Värtaverket power station,retired,2020.0,1990,141.0,L103529,0,Sweden,Europe & Central Asia,SWE


In [15]:
# drop 23 retirement plants without year
pw_data_coal = pw_data_coal[pw_data_coal["Coal_EndYr"]!="not found"]

In [16]:
pw_data_coal["Region"].unique()

array(['East Asia & Pacific', 'Sub-Saharan Africa',
       'Latin America & Caribbean', 'South Asia', 'North America',
       'Middle East & North Africa', 'Europe & Central Asia'],
      dtype=object)

In [17]:
pw_data_coal = pw_data_coal[pw_data_coal["Region"].isin(["Sub-Saharan Africa","Latin America & Caribbean","Middle East & North Africa"])==False]
pw_data_coal

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,Region,Country Code
0,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,1,New Zealand,East Asia & Pacific,NZL
1,-33.212081,151.542216,Munmorah power station,retired,2012.0,1969,600.0,L100008,1,Australia,East Asia & Pacific,AUS
2,-32.198337,115.775155,Kwinana power station,retired,2015.0,1970,640.0,L100041,1,Australia,East Asia & Pacific,AUS
3,-27.660536,152.812899,Swanbank-A power station,retired,2012.0,1966,396.0,L405135,1,Australia,East Asia & Pacific,AUS
4,-38.386550,144.182062,Anglesea power station,retired,2015.0,1969,160.0,L100031,0,Australia,East Asia & Pacific,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...
1077,48.552500,21.972625,Vojany I power station,retired,2010.0,1965,440.0,L103410,0,Slovakia,Europe & Central Asia,SVK
1078,48.698823,18.533458,Novaky power station,retired,2015.0,1957,252.0,L103415,0,Slovakia,Europe & Central Asia,SVK
1079,49.059218,18.907019,Martinska power station,retired,2020.0,1962,32.0,L103416,0,Slovakia,Europe & Central Asia,SVK
1080,59.353269,18.100901,Värtaverket power station,retired,2020.0,1990,141.0,L103529,0,Sweden,Europe & Central Asia,SWE


In [22]:
pw_data_all = pw_data_coal.copy()
for ind in pw_data_all.index:
    shift_year = int(float(pw_data_all.loc[ind,"Coal_EndYr"]))
    if shift_year>2021:
        shift_year = 2021
    shift_region = pw_data_all.loc[ind,"Region"]
    shift_country = pw_data_all.loc[ind,"GEM_Name"]
    if (shift_region == "East Asia & Pacific" and shift_country!= "China") or shift_region == "South Asia":
        pw_data_all.loc[ind,"Coal_Price_Var"] = coal_price.loc[shift_year,"Japan steam spot CIF price  †"]
        pw_data_all.loc[ind,"NG_Price_Var"] = ng_price.loc[shift_year,"Japan"]
    elif shift_country == "China":
        pw_data_all.loc[ind,"Coal_Price_Var"] = coal_price.loc[shift_year,"China Qinhuangdao spot price*"]
        pw_data_all.loc[ind,"NG_Price_Var"] = ng_price.loc[shift_year,"Japan"]
    elif shift_region == "Europe & Central Asia":
        if shift_year < 1987:
            shift_year = 1987
        pw_data_all.loc[ind,"Coal_Price_Var"] = coal_price.loc[shift_year,"Northwest Europe marker price †"]
        pw_data_all.loc[ind,"NG_Price_Var"] = ng_price.loc[shift_year,"Average German"]
    elif shift_region == "North America":
        if shift_year < 1990:
            shift_year = 1990
        pw_data_all.loc[ind,"Coal_Price_Var"] = coal_price.loc[shift_year,"US Central Appalachian coal spot price index ‡"]
        pw_data_all.loc[ind,"NG_Price_Var"] = ng_price.loc[shift_year,"US"]
pw_data_all["Coal_Price_Var"] = pw_data_all["Coal_Price_Var"]/27.78 # convert the unit to million BTU from tonne of coal
pw_data_all["GasMinusCoal_Var"] = pw_data_all["NG_Price_Var"] - pw_data_all["Coal_Price_Var"]
pw_data_all

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,Region,Country Code,Coal_Price_Var,NG_Price_Var,GasMinusCoal_Var
0,-37.544470,175.148805,Huntly power station,retired,2013.0,1983,500.0,L103096,1,New Zealand,East Asia & Pacific,NZL,3.242081,16.174162,12.932082
1,-33.212081,151.542216,Munmorah power station,retired,2012.0,1969,600.0,L100008,1,Australia,East Asia & Pacific,AUS,3.610451,16.753873,13.143422
2,-32.198337,115.775155,Kwinana power station,retired,2015.0,1970,640.0,L100041,1,Australia,East Asia & Pacific,AUS,2.163279,10.266579,8.103300
3,-27.660536,152.812899,Swanbank-A power station,retired,2012.0,1966,396.0,L405135,1,Australia,East Asia & Pacific,AUS,3.610451,16.753873,13.143422
4,-38.386550,144.182062,Anglesea power station,retired,2015.0,1969,160.0,L100031,0,Australia,East Asia & Pacific,AUS,2.163279,10.266579,8.103300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,48.552500,21.972625,Vojany I power station,retired,2010.0,1965,440.0,L103410,0,Slovakia,Europe & Central Asia,SVK,3.329720,8.031026,4.701306
1078,48.698823,18.533458,Novaky power station,retired,2015.0,1957,252.0,L103415,0,Slovakia,Europe & Central Asia,SVK,2.044422,6.719085,4.674664
1079,49.059218,18.907019,Martinska power station,retired,2020.0,1962,32.0,L103416,0,Slovakia,Europe & Central Asia,SVK,1.805769,4.062832,2.257063
1080,59.353269,18.100901,Värtaverket power station,retired,2020.0,1990,141.0,L103529,0,Sweden,Europe & Central Asia,SWE,1.805769,4.062832,2.257063


### Add other indicators

In [19]:
lng = pd.read_csv("../../data/global_GEM/analysis_infra/LNG_terminal_operating.csv")
lng = lng[lng['StartYear1'].isna()==False]
lng_import = lng[lng["Import/Export"]=="Import"]
lng_import

,TerminalID,ProjectID,ComboID,Country,Region,Wiki,TerminalName,UnitName,OtherEnglishNames,Owner,...,State/Province,Latitude,Longitude,Accuracy,Floating,FID,FIDYear,OtherLanguageName,PowerPlantsSupplied,OtherLanguageWikiPage
19,T0217,0,T021700,United States,North America,https://www.gem.wiki/Cove_Point_LNG_Terminal,Cove Point LNG Import Terminal,NaN,NaN,"Cove Point LNG, LP [200.00%]",...,Maryland,38.391342,-76.404508,exact,NaN,NaN,NaN,NaN,NaN,NaN
31,T0240,0,T024000,United States,North America,https://www.gem.wiki/Sabine_Pass_LNG_Terminal,Sabine Pass LNG Import Terminal,NaN,NaN,Sabine Pass LNG [100.00%],...,Louisiana,29.754097,-93.874051,exact,NaN,NaN,NaN,NaN,NaN,NaN
68,T0270,0,T027000,Bangladesh,South Asia,https://www.gem.wiki/Moheshkhali_FLNG_Terminal...,Moheshkhali FLNG Terminal (Petrobangla),NaN,MLNG,Petrobangla [100.00%],...,Chittagong,21.550000,91.950000,approximate,yes,NaN,NaN,NaN,NaN,NaN
69,T0272,0,T027200,Bangladesh,South Asia,https://www.gem.wiki/Summit_FSRU,Summit FSRU,NaN,NaN,Summit Oil and Shipping Co Ltd [75.00%]; Mitsu...,...,Chittagong,21.550000,91.950000,approximate,yes,NaN,NaN,NaN,NaN,NaN
75,T0275,0,T027500,Indonesia,SE Asia,https://www.gem.wiki/Arun_LNG_Terminal,Arun LNG Import Terminal,NaN,NaN,Pertamina [70.00%]; Government of Aceh [30.00%],...,Sumatra,5.223400,97.083000,exact,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,T0876,0,T087600,Japan,East Asia,https://www.gem.wiki/Toyama_Shinko_LNG_Terminal,Toyama Shinko LNG Terminal,NaN,NaN,Hokuriku Electric [100.00%],...,NaN,36.757158,137.222460,approximate,NaN,NaN,NaN,NaN,NaN,NaN
336,T0882,0,T088200,Myanmar,SE Asia,https://www.gem.wiki/Thaketa_LNG-to-Power_LNG_...,Thaketa LNG-to-Power LNG Terminal,NaN,NaN,Siemens Energy [unknown %]; TotalEnergies [unk...,...,NaN,16.788223,96.218176,approximate,NaN,NaN,NaN,NaN,NaN,NaN
337,T0908,0,T090800,Azerbaijan,Eurasia,https://www.gem.wiki/Sangachal_LNG_Terminal,Sangachal LNG Terminal,NaN,NaN,BP [100.00%],...,NaN,40.198128,49.478162,approximate,NaN,NaN,NaN,NaN,NaN,NaN
339,T0925,0,T092500,Brazil,Latin America and the Caribbean,https://www.gem.wiki/Bahia_FSRU,Bahia FSRU,NaN,NaN,Petrobras [100.00%],...,Bahia,-12.785422,-38.703511,approximate,yes,NaN,NaN,FSRU da Bahia,NaN,https://www.gem.wiki/FSRU_da_Bahia


In [42]:
columns_to_keep = ["Country Code", "BP_Name", "Hanson_Name", "Reform_Name"]
country_name = country_name[columns_to_keep]
combined_data1 = pd.merge(pw_data_all,country_name,on="Country Code")
combined_data1["Coal_EndYr"] = combined_data1["Coal_EndYr"].astype(float).astype(int)
combined_data1 = combined_data1.sort_values(by="Coal_EndYr")
combined_data1 = pd.merge(combined_data1,reform_data,left_on=["Coal_EndYr","Reform_Name"],right_on=["year","country"],how="left")
combined_data1 = combined_data1.drop(columns=["year","source","country"])
combined_data1 = pd.merge(combined_data1,BP_data,left_on=["Coal_EndYr","BP_Name"],right_on=["year","country"],how="left")
combined_data1[BP_data.columns[2:]] = combined_data1[BP_data.columns[2:]].fillna(0) # those without data, assume 0
combined_data1 = combined_data1.drop(columns=["year","country"])
combined_data1 = pd.merge(combined_data1,world_dev_data,left_on=["Coal_EndYr","Country Code"],right_on=["year","Country Code"],how="left")
combined_data1

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,...,Population,WDI_EnergyGDP_kgOilEq/$1k,WDI_Manu_GDP_%,WDI_CoalRents_%,WDI_OilRents_%,WDI_NGRents_%,WDI_Coal_El_%,WDI_NG_El_%,WDI_Fossil_El_%,logGDPpc
0,50.133377,30.746670,Trypilska power station,retired,1971,1972,600.0,L103742,1,Ukraine,...,47597756.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50.133377,30.746670,Trypilska power station,retired,1972,1972,600.0,L103742,1,Ukraine,...,47974187.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54.035556,39.779444,NaN,NaN,1984,1973,1600.0,NaN,1,Russia,...,142745000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55.656850,12.556960,NaN,NaN,1994,1920,25.0,NaN,1,Denmark,...,5206180.0,89.602219,14.449147,0.000000,0.294240,0.045226,5.618808,82.039529,95.132831,10.644703
4,53.231700,-3.081490,NaN,NaN,1996,1954,180.0,NaN,1,United Kingdom,...,58166950.0,114.029458,15.094408,0.006024,0.692863,0.148572,24.071821,42.159610,71.151088,10.434843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,42.382408,-87.813586,Waukegan Generating Station,retired,2022,1952,802.7,L103875,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043,42.122192,-83.180975,Trenton Channel Power Plant,retired,2022,1949,775.5,L103991,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1044,37.015273,117.411912,Shandong Port Public Heat Supply power station,retired,2022,not found,195.0,L104646,0,China,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,21.303111,-158.108050,AES Hawaii Generation Plant,retired,2022,1992,203.0,L103851,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
"""
columns_to_keep = ["Country Code", "BP_Name", "Hanson_Name", "Reform_Name"]
country_name = country_name[columns_to_keep]
combined_data1 = pd.merge(pw_data_all,country_name,on="Country Code")
combined_data1["BP_Name"] = combined_data1["BP_Name"].fillna("mark")
power_col = power_reform.columns[3:]
combined_data1["Coal_EndYr"] = combined_data1["Coal_EndYr"].astype(float).astype(int)
combined_data1 = combined_data1.sort_values(by="Coal_EndYr")

for ind in combined_data1.index:
    ind_year = int(float(combined_data1.loc[ind,"Coal_EndYr"]))
    ind_country = combined_data1.loc[ind,"BP_Name"]
    ind_country_wb = combined_data1.loc[ind,"Country Code"]
    ind_country_hanson = combined_data1.loc[ind,"Hanson_Name"]
    ind_country_ref = combined_data1.loc[ind,"Reform_Name"]
    
    # coal rent variable
    if ind_year > 2020 or ind_year < 1980:
        combined_data1.loc[ind,"CoalRent_Var"] = np.nan
    else:
        ind_year_r2p = ind_year
        coal_rent1 = fossil_rents_ts[fossil_rents_ts['Series Name']=='Coal rents (% of GDP)']
        coal_rent = coal_rent1[coal_rent1['Country Code']==ind_country_wb][str(ind_year_r2p)]
        combined_data1.loc[ind,"CoalRent_Var"] = coal_rent.iloc[0]
    
    # NG reserve to production 
    if ind_country == "mark":
        ng_var = 0
    else:
        ng_var = ng_r2p.loc[ind_country,str(ind_year_r2p)] 
    combined_data1.loc[ind,"NG_R2P_Var"] = ng_var
    
    # state capacity
    this_country_hanson = state_capacity[state_capacity["country"]==ind_country_hanson]
    hanson_year_list = list(this_country_hanson["year"])
    if (float(ind_year) in hanson_year_list) == False:
        combined_data1.loc[ind,"State_Capacity_Var"] = np.nan
    else:
        combined_data1.loc[ind,"State_Capacity_Var"] = this_country_hanson[this_country_hanson["year"]==float(ind_year)]["Capacity"].iloc[0]
        
    # power reform
    this_country_reform = power_reform[power_reform["cntry"]==ind_country_ref]
    reform_year_list = list(this_country_reform["year"])
    if (float(ind_year) in reform_year_list) == False:
        combined_data1.loc[ind,power_col+"_Var"] = np.nan
    else:
        combined_data1.loc[ind,power_col+"_Var"] = this_country_reform[this_country_reform["year"]==float(ind_year)].loc[:,power_col].values

combined_data1.head(20)
"""

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,...,NG_R2P_Var,State_Capacity_Var,R_IndepProducer_Var,R_Private_Var,R_Unbundle_Var,R_WholeSale_Var,R_IndepReg_Var,R_Choice_Var,R_Liberalization_Var,R_Corp_Var
976,50.133377,30.746670,Trypilska power station,retired,1971,1972,600.0,L103742,1,Ukraine,...,57.079159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
977,50.133377,30.746670,Trypilska power station,retired,1972,1972,600.0,L103742,1,Ukraine,...,57.079159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,54.035556,39.779444,NaN,NaN,1984,1973,1600.0,NaN,1,Russia,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1019,55.656850,12.556960,NaN,NaN,1994,1920,25.0,NaN,1,Denmark,...,25.410006,2.85700,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,53.231700,-3.081490,NaN,NaN,1996,1954,180.0,NaN,1,United Kingdom,...,8.414644,1.83700,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
806,51.241256,58.497233,NaN,NaN,1998,1957,195.0,NaN,1,Russia,...,61.638453,0.05997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
954,52.749600,15.268700,Gorzow power station,retired,1999,1976,30.0,L103226,1,Poland,...,29.844644,1.28100,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
817,55.008713,82.853682,Novosibirsk-3 power station,retired,2000,1952,150.0,L103365,0,Russia,...,61.736039,0.31450,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,41.070000,-8.458700,Tapada do Outeiro power station,retired,2000,1964,50.0,L104725,1,Portugal,...,0.000000,1.69300,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
926,53.487278,9.508231,Hafen Hamburg power station,retired,2000,1981,80.0,L104706,0,Germany,...,13.135975,1.90800,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [37]:
def calc_distance(lat1,lon1,lat2,lon2):
    
    # Approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
"""
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print geopy.distance.geodesic(coords_1, coords_2).km
"""

In [43]:
# add LNG terminal distance

combined_data2 = combined_data1.reset_index().drop(columns="index")
for ind in combined_data2.index:
    coal_end_year = combined_data2.loc[ind,"Coal_EndYr"]
    lat1 = combined_data2.loc[ind,"Coal_Latitude"]
    lon1 = combined_data2.loc[ind,"Coal_Longitude"]
    
    for port in ["Import","Export"]:
        
        lng_port = lng[lng["Import/Export"]==port]
        exist_lng = lng_port[lng_port["StartYear1"]<coal_end_year]
        for ind2 in exist_lng.index:
            lat2 = exist_lng.loc[ind2,"Latitude"]
            lon2 = exist_lng.loc[ind2,"Longitude"]
            dist = calc_distance(lat1,lon1,lat2,lon2)
            exist_lng.loc[ind2,"DISTANCE"] = dist
        if len(exist_lng)==0:
            combined_data2.loc[ind,"LNG_"+port+"_km"] = np.nan
        else:
            exist_lng = exist_lng.sort_values(by="DISTANCE")
            combined_data2.loc[ind,"LNG_"+port+"_km"] = exist_lng["DISTANCE"].iloc[0]
combined_data2

,Coal_Latitude,Coal_Longitude,Coal_Plant,Coal_Status,Coal_EndYr,Coal_StartYr,Coal_MW,Coal_TrackerLOC,Transition,GEM_Name,...,WDI_Manu_GDP_%,WDI_CoalRents_%,WDI_OilRents_%,WDI_NGRents_%,WDI_Coal_El_%,WDI_NG_El_%,WDI_Fossil_El_%,logGDPpc,LNG_Import_km,LNG_Export_km
0,50.133377,30.746670,Trypilska power station,retired,1971,1972,600.0,L103742,1,Ukraine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2406.172395,NaN
1,50.133377,30.746670,Trypilska power station,retired,1972,1972,600.0,L103742,1,Ukraine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1713.596781,8699.231474
2,54.035556,39.779444,NaN,NaN,1984,1973,1600.0,NaN,1,Russia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2423.080176,3389.725123
3,55.656850,12.556960,NaN,NaN,1994,1920,25.0,NaN,1,Denmark,...,14.449147,0.000000,0.294240,0.045226,5.618808,82.039529,95.132831,10.644703,780.253071,2412.444847
4,53.231700,-3.081490,NaN,NaN,1996,1954,180.0,NaN,1,United Kingdom,...,15.094408,0.006024,0.692863,0.148572,24.071821,42.159610,71.151088,10.434843,476.841534,1951.074642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,42.382408,-87.813586,Waukegan Generating Station,retired,2022,1952,802.7,L103875,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1062.494696,1062.494696
1043,42.122192,-83.180975,Trenton Channel Power Plant,retired,2022,1949,775.5,L103991,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,708.917761,708.917761
1044,37.015273,117.411912,Shandong Port Public Heat Supply power station,retired,2022,not found,195.0,L104646,0,China,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.684539,2354.186936
1045,21.303111,-158.108050,AES Hawaii Generation Plant,retired,2022,1992,203.0,L103851,0,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4242.779179,4373.899773


In [44]:
combined_data2.to_csv("coal2gas_plantwise.csv")